## Eulerian Fluid Simulation
*still sensors that never moves*
- Eulerian representation uses still sensors in space, usually arranged in a
 regular grid/triangular mesh.
- A bit of math - but not too much.

### Overview
#### Material Derivatives: Lagrangian v.s. Eulerian (材料导数，物质导数)

\begin{equation*}
\frac{D}{Dt} := \frac{\partial}{\partial t}+{\rm u}\cdot\nabla
\end{equation*}

E.g.,<br>
\begin{equation*}\begin{array}{rcl}
\displaystyle\frac{DT}{Dt} & = & \displaystyle\frac{\partial T}{\partial t}+{\rm u}\cdot\nabla T \\
\displaystyle\frac{D{\rm u}_{x}}{Dt} & = & \displaystyle\frac{\partial {\rm u}_{x}}{\partial t}+{\rm u}\cdot\nabla {\rm u}_{x}
\end{array}\end{equation*}<br>
${\rm u}$: material (fluid) velocity. Many other names: Advective/Lagrangian/particle derivative.
Intuitively, change of physical quantity on a piece of material =
1. change due to time $\frac{\partial}{\partial t}$ (Eulerian).
2. change due to material movement ${\rm u}\cdot\nabla$.